In [1]:
from transformers import AutoTokenizer, BartTokenizerFast, CodeGenTokenizerFast, PreTrainedTokenizerFast, LlamaTokenizerFast

In [2]:
new_repo = 'google-t5/t5-large'

bart_large_mnli_tok:BartTokenizerFast = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')
bart_large_tok:BartTokenizerFast = AutoTokenizer.from_pretrained('facebook/bart-large')
new_tok = AutoTokenizer.from_pretrained(new_repo)

# new_tok.add_eos_token = True
# new_tok.add_bos_token = True

# new_tok.pad_token = '<PAD>'



/mnt/c/Users/lachl/PycharmProjects/bart-large-mnli-recreation/.lvenv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
print(f"{type(bart_large_mnli_tok)=}")
print(f"{type(bart_large_tok)=}")
print(f"{type(new_tok)=}")

type(bart_large_mnli_tok)=<class 'transformers.models.bart.tokenization_bart_fast.BartTokenizerFast'>
type(bart_large_tok)=<class 'transformers.models.bart.tokenization_bart_fast.BartTokenizerFast'>
type(new_tok)=<class 'transformers.models.t5.tokenization_t5_fast.T5TokenizerFast'>


In [4]:
# Pad tokens used
print(f"{bart_large_mnli_tok.pad_token}")
print(f"{bart_large_tok.pad_token}")
print(f"{new_tok.pad_token}")# Pad tokens used# Pad tokens used

<pad>
<pad>
<pad>


In [5]:
# Sep tokens used
print(f"{bart_large_mnli_tok.sep_token}")
print(f"{bart_large_tok.sep_token}")
print(f"{new_tok.sep_token}")# Pad tokens used

</s>
</s>
None


In [6]:
print(f"{bart_large_mnli_tok.model_max_length}")
print(f"{bart_large_tok.model_max_length}")
print(f"{new_tok.model_max_length}")

1024
1024
1000000000000000019884624838656


In [7]:

premises = [
    "and",
]

hypotheses = [
    'my'
]

def apply_tokenizer(tokenizer:PreTrainedTokenizerFast, premises, hypotheses, also_decode=True):
    
    outputs = tokenizer(premises, hypotheses, truncation='only_first', padding="max_length",
                        max_length=25, add_special_tokens=True) 
    
    if also_decode:
        outputs['decoded_input_ids'] = [[tokenizer.decode(i) for i in seq] for seq in outputs['input_ids']]
    
    return outputs


In [8]:
apply_tokenizer(bart_large_mnli_tok, premises, hypotheses)

{'input_ids': [[0, 463, 2, 2, 4783, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'decoded_input_ids': [['<s>', 'and', '</s>', '</s>', 'my', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']]}

In [9]:
apply_tokenizer(bart_large_tok, premises, hypotheses)

{'input_ids': [[0, 463, 2, 2, 4783, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'decoded_input_ids': [['<s>', 'and', '</s>', '</s>', 'my', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']]}

In [10]:
# this currently throws an error because we have no padding token, so let's add it!
# apply_tokenizer(new_tok, premises, hypotheses)

In [11]:
# add pad token to new tokenizer and model

from transformers import AutoModelForSequenceClassification

new_model = AutoModelForSequenceClassification.from_pretrained(new_repo, num_labels=3)

def add_special_tokens_when_missing(tokenizer, model):
    
    
    needs_pad_token = tokenizer.pad_token is None
    if needs_pad_token:
        num_added_toks = tokenizer.add_special_tokens({'pad_token': '<|pad_token|>'})
        assert num_added_toks == 1

    needs_sep_token = tokenizer.sep_token is None
    if needs_sep_token:
        num_added_toks = tokenizer.add_special_tokens({'sep_token': '<|sep_token|>'})
        assert num_added_toks == 1

    needs_eos_token = tokenizer.eos_token is None
    if needs_eos_token:
        num_added_toks = tokenizer.add_special_tokens({'eos_token': '<|eos_token|>'})
        assert num_added_toks == 1

    needs_bos_token = tokenizer.bos_token is None
    if needs_eos_token:
        num_added_toks = tokenizer.add_special_tokens({'bos_token': '<|bos_token|>'})
        assert num_added_toks == 1

    if any([needs_pad_token, needs_sep_token, needs_eos_token, needs_bos_token]):
        model.resize_token_embeddings(len(tokenizer))
        
    return tokenizer, model
        
    
new_tok, new_model = add_special_tokens_when_missing(new_tok, new_model)

# print(f"{type(new_model)=}")

model.safetensors:  30%|###       | 891M/2.95G [00:00<?, ?B/s]

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google-t5/t5-large and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
apply_tokenizer(new_tok, premises, hypotheses)

{'input_ids': [[11, 1, 82, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'decoded_input_ids': [['and', '</s>', 'my', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']]}

In [13]:

parent_class = type(new_tok)

class UpdatedTokenizerFast(parent_class):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):

        if token_ids_1 is None: 
            raise NotImplementedError('This method is designed for zero shot classification, so requires a premise and hypothesis to be passed. No token_ids_1 was passed.')

        output = [self.bos_token_id] + token_ids_0 + [self.sep_token_id] + token_ids_1 + [self.eos_token_id]

        return output
    
updated_new_tok = UpdatedTokenizerFast.from_pretrained(new_repo)

updated_new_tok, new_model = add_special_tokens_when_missing(updated_new_tok, new_model)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5TokenizerFast'. 
The class this function is called from is 'UpdatedTokenizerFast'.


In [14]:
# this currently throws an error because we have no padding token, so let's add it!
# apply_tokenizer(updated_new_tok, premises, hypotheses)

In [15]:

outputs = new_tok(
    premises, 
    hypotheses, 
    truncation='only_first', 
    max_length=25,
    padding="max_length", 
    add_special_tokens=True
) 

print(outputs)

{'input_ids': [[11, 1, 82, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}


In [16]:

decoded_input_ids = [new_tok.decode(i) for i in outputs]

print(decoded_input_ids)

TypeError: argument 'ids': Can't extract `str` to `Vec`